In [ ]:
import os
iskaggle = os.environ.get('KAGGLE_KERNEL_RUN_TYPE','')

if iskaggle:
  !pip install -Uqq fastai ' duckduckgo_search>=6.2'

In [ ]:
!pip install -U duckduckgo_search

In [ ]:
from duckduckgo_search import DDGS
from fastcore.all import*

def search_images(keywords,max_images=200):
   return L(DDGS().images(keywords, max_results = max_images)).itemgot('image')
import time,json

In [ ]:
urls = search_images('dig photoes', max_images = 1)
urls[0]

In [ ]:
from fastdownload import download_url
dest = 'dog.jpg'
download_url(urls[0], dest, show_progress=False)

from fastai.vision.all import *
im = Image.open(dest)

im.to_thumb(   256,256)

In [ ]:
download_url(search_images("cats photos", max_images = 1)[0], 'cat.jpg', show_progress = False)
Image.open('cat.jpg').to_thumb(256,256)

In [ ]:

searches = 'dog','cat'
path = Path('cat_or_not_cat')

for o in searches:
    dest = (path/o)
    dest.mkdir(exist_ok=True, parents=True)
    download_images(dest, urls=search_images(f'{o} photo'))
    time.sleep(5)
    resize_images(path/o, max_size=400, dest=path/o)

In [ ]:
failed = verify_images(get_image_files(path))
failed.map(Path.unlink)
len(failed)

In [ ]:
dls = DataBlock(
    blocks=(ImageBlock, CategoryBlock),
    get_items=get_image_files,
    splitter=RandomSplitter(valid_pct=0.2, seed=42),
    get_y=parent_label,
    item_tfms=[Resize(192, method='squish')]
).dataloaders(path, bs=32)

dls.show_batch(max_n=6)

In [ ]:
learn = vision_learner(dls, resnet18, metrics = error_rate)
learn.fine_tune(3, 0.01)

In [ ]:
is_bird,_,probs = learn.predict(PILImage.create('dog.jpg'))
print(f"This is a: {is_bird}.")
print(f"Probability it's a bird: {probs[0]:.4f}")